In [1]:
import pandas as pd
import re
import msoffcrypto
import io
import openpyxl
import os
import shutil
import requests
import json
import datetime
import itertools
import pickle
import operator


from xlrd import XLRDError
from zipfile import BadZipFile
from pathlib import Path
import dateparser
from collections import Counter, defaultdict


In [2]:
def save_dfs(dfs: dict[str, pd.DataFrame], path):
    """
    Сохраняет pandas datafram-ы dfs
    в frt файлы в папке path
    """
    Path(path).mkdir(parents=True, exist_ok=True)
    for path_to_file, df in dfs.items():
        df.to_feather(path=f"{path}/{os.path.basename(path_to_file)}_parsed")


def read_files(dir, regex = r".*"):
    """
    Читает excel файлы в папке и подпапках dir,
    имена которых удовлетворяют regex
    """
    skipped_files = []
    errors = set()

    def handler(path, exc):
        if type(exc) == XLRDError and str(exc) == "Excel xlsx file; not supported":
            return pd.read_excel(path, engine='openpyxl')
        skipped_files.append((str(path), exc.__repr__()))
        errors.add(exc.__repr__())


    def wrapper(dir):
        files = dict()
        regex_engine = [
            (r".*xlsx",'openpyxl'),
            (r".*xls", 'xlrd'),
        ]
        for file_or_dir in dir.iterdir():
            if file_or_dir.is_dir():
                files.update(wrapper(file_or_dir))
            for regex_excel, engine in regex_engine:
                if re.fullmatch(regex_excel, str(file_or_dir)) and re.fullmatch(regex, str(file_or_dir)):
                    try:
                        files[str(file_or_dir)] = pd.read_excel(file_or_dir, engine=engine)
                    except Exception as e:
                        result = handler(file_or_dir, e)
                        if result is not None:
                            files[str(file_or_dir)] = result
        return files
    result = wrapper(dir)
    print(f"Unparsed files: ", *skipped_files, sep='\n')
    print(f"Errors: ", *errors)
    return result



In [3]:
def parse_date(series):
    """
    Парсит нормальные даты и даты типа 'Июнь 2020'
    """
    for i, elem in enumerate(series):
        series.loc[i] = dateparser.parse(elem)
    return series


def extract_year(name):
    """
    Извлекает год из имени файла - 1
    """
    start, _ = re.search(r"01.01.20.*xls.*", name).span()
    return int(name[start + 6: start + 10]) - 1

def extract_string_for_search(s):
    """
    Извлекает подстроку, которая начинается
    с 'г.' или 'город'
    """
    city = re.search("(г\.|город ).*", s)

    if city is not None:
        start, end = city.span()
        return s[start:end]

    return s

def add_string_col_to_df(df, raw_string_column="raw_string", result_column="clean_address"):
    """
    Применяет extract_string_for_search к столбцу df[raw_string_column]
    и сохраняет результат в df[result_column]
    """
    df[result_column] = df[raw_string_column].apply(extract_string_for_search)
    return df


In [4]:
def process_kr(df):
    """
    Обрабатывает kr df за 2020, 2021, 2022, 2023
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Общая стоимость",
        "Начало",
        "Окончание",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Код состояния проекта/работы",
        "Документ",
        "Дата",
        ]
    columns_cast_to_str = [
        "Начало",
        "Окончание",
    ]
    for column in columns_cast_to_str:
        df[column] = df[column].astype("string")
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = parse_date(df["Начало"].astype(str))
    df["end_date"] =  parse_date(df["Окончание"].astype(str))
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_kr_old(df, name):
    """
    Обрабатывает kr файлы за 2017, 2018
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "№ п/п",
        "Наименование раздела, наименование территориального учреждения, другого подразделения Банка",
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "По объектам целевого резерва",
        "Освоено",
        "Ожидаемое освоение за год",
        "Дата завершения работ (месяц, год)",
        ]
    columns_with_amount_of_money = [
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего (утверждено)",
        "В том числе по статьям (утверждено)",
        "По объектам целевого резерва",
        "Освоено",
        "Ожидаемое освоение за год",
    ]
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(9)])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование раздела, наименование территориального учреждения, другого подразделения Банка"]
    df["start_date"] = pd.to_datetime(extract_year(name))
    df["end_date"] =  parse_date(df["Дата завершения работ (месяц, год)"].astype(str))
    df["amount_money"] = pd.to_numeric(df["Всего (утверждено)"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_kr_2019(df, name):
    """
    Обрабатывает kr файлы за 2019
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "№ п/п",
        "Наименование раздела, наименование территориального учреждения, другого подразделения Банка России, наименование и местонахождение объекта",
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего(утверждено)",
        "по объектам, обеспеченным документацией",
        "по объектам целевого резерва",
        "Освоено за отчетный период",
        "Ожидаемое освоение за год",
        "Дата завершения работ (месяц, год)",
        ]
    columns_with_amount_of_money = [
        "Сметная стоимость капитального ремонта, в т.ч. ПИР",
        "Освоено",
        "Всего(утверждено)",
        "по объектам, обеспеченным документацией",
        "по объектам целевого резерва",
        "Освоено за отчетный период",
        "Ожидаемое освоение за год",
    ]
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(9)])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование раздела, наименование территориального учреждения, другого подразделения Банка России, наименование и местонахождение объекта"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = parse_date(df["Дата завершения работ (месяц, год)"].astype(str))
    df["amount_money"] = pd.to_numeric(df["Всего(утверждено)"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_ks(df):
    """
    Обрабатывает ks файлы за 2020, 2021, 2022, 2023
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Общая стоимость",
        "Начало",
        "Окончание",
        "Всего (утверждено)",
        "В том числе по статьям 1 (утверждено)",
        "В том числе по статьям 2 (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям 1 (на отчетную дату)",
        "В том числе по статьям 2 (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям 1 (ожидаемое исполнение)",
        "В том числе по статьям 2 (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Код состояния проекта/работы",
        "Документ",
        "Дата",
        ]
    columns_with_amount_of_money = [
       "Всего (утверждено)",
        "В том числе по статьям 1 (утверждено)",
        "В том числе по статьям 2 (утверждено)",
        "Всего (на отчетную дату)",
        "В том числе по статьям 1 (на отчетную дату)",
        "В том числе по статьям 2 (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "В том числе по статьям 1 (ожидаемое исполнение)",
        "В том числе по статьям 2 (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
    ]
    for column in columns_with_amount_of_money:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1, 2, 3])
    df = df.reset_index(drop=True)
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = parse_date(df["Начало"].astype(str))
    df["end_date"] =  parse_date(df["Окончание"].astype(str))
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])
    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_ks_old(df):
    """
    Обрабатывает kr файлы за 2017, 2018, 2019
    """
    df = df.drop(df.columns[-1], axis = 1)
    columns = df.columns = [
        "Номер проекта/ работы",
        "Наименование проекта/работы",
        "Наименование подразделения Банка России",
        "Начало",
        "Окончание",
        "Общая стоимость",
        "Всего (освоено)",
        "В том числе по статьям незавершеннное строительство",
        "Всего (план)",
        "В том числе по объектам, обеспеченным документацией",
        "В том числе по объектам целевого резерва",
        "Всего (на отчетную дату)",
        "% от годового объема затрат (расходов) по актуальной версии плана (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "% от годового объема затрат (расходов) по актуальной версии плана (ожидаемое исполнение)",
        "Объем фондов",
        "Срок ввода фондов (месяц, год)",
    ]
    columns_with_amount_of_money = [
        "Общая стоимость",
        "Всего (освоено)",
        "В том числе по статьям незавершеннное строительство",
        "Всего (план)",
        "В том числе по объектам, обеспеченным документацией",
        "В том числе по объектам целевого резерва",
        "Всего (на отчетную дату)",
        "Всего (ожидаемое исполнение)",
        "Объем фондов",
    ]
    df = df[df[columns[0]].isna() == False]
    df = df.reset_index(drop=True)
    df = df.drop([0, 1, 2, 3])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] = pd.to_numeric(df[column], errors='coerce')
        df[column] *= 1000
    df["raw_string"] = df["Наименование проекта/работы"]
    df["start_date"] = pd.to_datetime(df["Начало"], format='mixed')
    df["end_date"] = pd.to_datetime(df["Окончание"], format='mixed')
    df["amount_money"] = pd.to_numeric(df["Общая стоимость"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_tr(df, name):
    """
    Обрабатывает tr файлы за 2021, 2023
    """
    df.columns = [
        "Номер п/п",
        "Местонахождение (адрес) объекта",
        "Код строки",
        "Идентификационный номер закупки по ГПЗ",
        "Сумма сметных ассигнований по Справке №1",
        "Подрядная организация",
        "№ договора, дата кем подписан",
        "Сумма договора",
        "Срок действия договора (месяц начала и месяц окончания работ MM.ГГ/MM.ГГ)",
        "Выбор подрядчика",
        "Согласование договора",
        "Заключение договора",
        "Примечания",
        "хз",
    ]
    columns_with_amount_of_money = [
        "Сумма сметных ассигнований по Справке №1",
        "Сумма договора",
    ]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(13)])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] *= 1000

    df["raw_string"] = df["Местонахождение (адрес) объекта"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=12, day=31))
    df["amount_money"] = pd.to_numeric(df["Сумма сметных ассигнований по Справке №1"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

def process_tr_old(df, name):
    """
    Обрабатывает kr файлы за 2019, 2020, 2022
    """
    df.columns = [
        "Номер п/п",
        "Наименование статей и подстатей и местонахождение объектов",
        "Код строки",
        "Утверждено согласно справке по форме 1",
        "Запланировано по объектам согласно справке по форме 1",
        "Запланировано по объектам c учетом корректировки справки по форме 1",
        "Доведено на отчетную дату",
        "Освоено с начала года",
        "Подлежит освоению",
        "Код ТУ",
        ]
    columns_with_amount_of_money = [
        "Утверждено согласно справке по форме 1",
        "Запланировано по объектам согласно справке по форме 1",
        "Запланировано по объектам c учетом корректировки справки по форме 1",
        "Доведено на отчетную дату",
        "Освоено с начала года",
        "Подлежит освоению",
    ]
    df = df.reset_index(drop=True)
    df = df.drop([i for i in range(13)])
    df = df.reset_index(drop=True)
    for column in columns_with_amount_of_money:
        df[column] *= 1000
    df["raw_string"] = df["Наименование статей и подстатей и местонахождение объектов"]
    df["start_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=1, day=1))
    df["end_date"] = pd.to_datetime(datetime.date(year=extract_year(name), month=12, day=31))
    df["amount_money"] = pd.to_numeric(df[ "Запланировано по объектам согласно справке по форме 1"])

    return df[["raw_string", "start_date", "end_date", "amount_money"]]

In [5]:
def get_kr_joined(path):
    """
    Create final kr dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_kr = dict()
    dfs_kr_new = read_files(Path(path), f"{path}/KR.*202[0|1|2|3].*")
    dfs_kr.update({path: process_kr(df) for path, df in dfs_kr_new.items()})

    dfs_kr_old = read_files(Path(path), f"{path}/KR.*201[7|8].x.*")
    dfs_kr.update({path: process_kr_old(df, path) for path, df in dfs_kr_old.items()})

    dfs_kr_2019 = read_files(Path(path), f"{path}/KR.*2019.*")
    dfs_kr.update({path: process_kr_2019(df, path) for path, df in dfs_kr_2019.items()})

    kr_joined = pd.concat(dfs_kr, axis=0, join="outer", ignore_index=True)
    kr_joined =  kr_joined[kr_joined["amount_money"] > 0]
    kr_joined = kr_joined.dropna(subset=["raw_string"])
    kr_joined = kr_joined.drop_duplicates().reset_index(drop=True)
    kr_joined
    return kr_joined


def get_ks_joined(path):
    """
    Create final ks dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_ks = dict()

    dfs_ks_new = read_files(Path(path), f"{path}/KS.*202[0|1|2|3].*")
    dfs_ks.update({name: process_ks(df) for name, df in dfs_ks_new.items()})

    dfs_ks_old = read_files(Path(path), f"{path}/KC_TR_KR/KS.*20[1].*")
    dfs_ks.update({name: process_ks_old(df) for name, df in dfs_ks_old.items()})

    ks_joined = pd.concat(dfs_ks, axis=0, join="outer", ignore_index=True)
    ks_joined =  ks_joined[ks_joined["amount_money"] > 0.0]
    ks_joined = ks_joined.dropna(subset=["raw_string"])
    ks_joined = ks_joined.drop_duplicates().reset_index(drop=True)
    return ks_joined


def get_tr_joined(path):
    """
    Create final tr dataframe
    with columns 
    'raw_string', 'start_date', 'end_date', 'amount_of_money'
    """
    dfs_tr = dict()

    dfs_tr_2023 = read_files(Path(path), f"{path}/TR.*202[1|3].xls.*")
    dfs_tr.update({name: process_tr(df, name) for name, df in dfs_tr_2023.items()})

    dfs_tr_new = read_files(Path(path), f"{path}/TR.*202[0|2].xls.*")
    dfs_tr.update({name: process_tr_old(df, name) for name, df in dfs_tr_new.items()})

    dfs_tr_old = read_files(Path(path), f"{path}/TR.*2019.xls.*")
    dfs_tr.update({name: process_tr_old(df, name) for name, df in dfs_tr_old.items()})

    tr_joined = pd.concat(dfs_tr, axis=0, join="outer", ignore_index=True)
    tr_joined =  tr_joined[tr_joined["amount_money"] > 0.0]
    tr_joined = tr_joined.dropna(subset=["raw_string"])
    tr_joined = tr_joined.drop_duplicates().reset_index(drop=True)
    
    return tr_joined



In [6]:
path = "train_data/KC_TR_KR_clean"
ks_joined = get_ks_joined(path)
tr_joined = get_tr_joined(path)
kr_joined = get_kr_joined(path)

Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 
Unparsed files: 
Errors: 


In [7]:
add_string_col_to_df(ks_joined, raw_string_column="raw_string", result_column="clean_address")
add_string_col_to_df(tr_joined)
add_string_col_to_df(kr_joined)

,raw_string,start_date,end_date,amount_money,clean_address
0,Выборочный капитальный ремонт здания Централь...,2020-04-01 00:00:00,2022-03-01 00:00:00,3942549.0,"г. Владивосток, ул. Океанский проспект, д.34"
1,Выборочный капитальный ремонт здания Централь...,2021-03-01 00:00:00,2021-04-01 00:00:00,97782.0,"г. Владивосток, Океанский проспект, д.34 в час..."
2,Выборочный капитальный ремонт нежилых помещен...,2021-03-01 00:00:00,2021-09-01 00:00:00,2907500.0,"г. Владивосток, ул. Светланская, д.73"
3,Выборочный капитальный ремонт помещения бокс-...,2019-06-01 00:00:00,2021-08-01 00:00:00,12784075.0,"г. Владивосток, ул. Светланская, 71"
4,Капитальный ремонт здания убежища (лит. А) Дал...,2019-08-01 00:00:00,2021-12-01 00:00:00,9465229.0,"г. Владивосток, ул. Семеновская, 29а"
...,...,...,...,...,...
80,Выборочный капитальный ремонт административног...,2018-01-01 00:00:00,2018-10-24 00:00:00,2618920.0,"г. Магадан, ул. Пушкина, д. 4"
81,Выборочный капитальный ремонт административног...,2018-01-01 00:00:00,2018-09-24 00:00:00,2467100.0,Выборочный капитальный ремонт административног...
82,Капитальный ремонт наружной канализации здания...,2018-01-01 00:00:00,2018-08-24 00:00:00,1405710.0,Капитальный ремонт наружной канализации здания...
83,Капитальный ремонт здания «Спальный корпус № 2...,2018-01-01 00:00:00,2018-12-24 00:00:00,14180990.0,Капитальный ремонт здания «Спальный корпус № 2...


In [8]:
ks_joined

,raw_string,start_date,end_date,amount_money,clean_address
0,Реконструкция отдельных помещений информатизац...,2019-11-01 00:00:00,2022-03-01 00:00:00,38969029.0,"г. Владивосток, Океанский проспект, д. 34"
1,Создание автоматической системы газового пожар...,2019-03-01 00:00:00,2021-09-01 00:00:00,9098392.0,Создание автоматической системы газового пожар...
2,Создание приточно-вытяжной вентиляции в здании...,2019-07-01 00:00:00,2021-04-01 00:00:00,2626707.0,Создание приточно-вытяжной вентиляции в здании...
3,Создание системы вентиляции в помещениях столо...,2018-04-01 00:00:00,2021-07-01 00:00:00,9250355.0,"г. Владивосток, ул. Светланская, д. 73"
4,Создание системы кондиционирования воздуха в п...,2020-04-01 00:00:00,2022-07-01 00:00:00,22064173.0,"г. Владивосток, пр-кт Океанский, 34"
5,"Создание системы вентиляции в РКЦ, пристройке ...",2021-04-01 00:00:00,2021-12-01 00:00:00,368168.0,"г. Уссурийск, ул. Некрасова, д. 102"
6,Создание системы автоматического газового пожа...,2017-07-01 00:00:00,2022-02-01 00:00:00,5443001.0,"г.Благовещенск, ул. Б. Хмельницкого, 52/2"
7,Создание системы кондиционирования администрат...,2017-07-01 00:00:00,2022-03-01 00:00:00,7423212.0,"г.Благовещенск, ул. Б. Хмельницкого, 52/2"
8,Создание системы приточно-вытяжной вентиляции ...,2017-07-01 00:00:00,2022-03-01 00:00:00,5894921.0,"г.Благовещенск, ул. Б. Хмельницкого, 52/2"
9,Создание автоматической установки газового пож...,2019-05-01 00:00:00,2021-11-01 00:00:00,9264732.0,"г.Биробиджан, проспект 60 -летия образования С..."


In [9]:
kr_joined

,raw_string,start_date,end_date,amount_money,clean_address
0,Выборочный капитальный ремонт здания Централь...,2020-04-01 00:00:00,2022-03-01 00:00:00,3942549.0,"г. Владивосток, ул. Океанский проспект, д.34"
1,Выборочный капитальный ремонт здания Централь...,2021-03-01 00:00:00,2021-04-01 00:00:00,97782.0,"г. Владивосток, Океанский проспект, д.34 в час..."
2,Выборочный капитальный ремонт нежилых помещен...,2021-03-01 00:00:00,2021-09-01 00:00:00,2907500.0,"г. Владивосток, ул. Светланская, д.73"
3,Выборочный капитальный ремонт помещения бокс-...,2019-06-01 00:00:00,2021-08-01 00:00:00,12784075.0,"г. Владивосток, ул. Светланская, 71"
4,Капитальный ремонт здания убежища (лит. А) Дал...,2019-08-01 00:00:00,2021-12-01 00:00:00,9465229.0,"г. Владивосток, ул. Семеновская, 29а"
...,...,...,...,...,...
80,Выборочный капитальный ремонт административног...,2018-01-01 00:00:00,2018-10-24 00:00:00,2618920.0,"г. Магадан, ул. Пушкина, д. 4"
81,Выборочный капитальный ремонт административног...,2018-01-01 00:00:00,2018-09-24 00:00:00,2467100.0,Выборочный капитальный ремонт административног...
82,Капитальный ремонт наружной канализации здания...,2018-01-01 00:00:00,2018-08-24 00:00:00,1405710.0,Капитальный ремонт наружной канализации здания...
83,Капитальный ремонт здания «Спальный корпус № 2...,2018-01-01 00:00:00,2018-12-24 00:00:00,14180990.0,Капитальный ремонт здания «Спальный корпус № 2...


In [10]:
tr_joined

,raw_string,start_date,end_date,amount_money,clean_address
0,"Нежилые помещения г. Владивосток, ул. Светланс...",2020-01-01,2020-12-31,259990.0,"г. Владивосток, ул. Светланская, 73а (51-01-01..."
1,"Нежилые помещения г. Владивосток, ул. Светланс...",2020-01-01,2020-12-31,122590.0,"г. Владивосток, ул. Светланская, 73а (51-01-01..."
2,"Жилое помещение, Приморский край, г. Владивост...",2020-01-01,2020-12-31,114940.0,"г. Владивосток, ул. Верхнепортовая, 44, кв.13 ..."
3,"Жилое помещение, Приморский край, г. Владивост...",2020-01-01,2020-12-31,333130.0,"г. Владивосток, ул. Верхнепортовая, 44, кв.135..."
4,Здание центрального банка Российской Федерации...,2020-01-01,2020-12-31,122090.0,Здание центрального банка Российской Федерации...
...,...,...,...,...,...
250,"11. Здание столовая на 108 мест, лит. Т пансио...",2018-01-01,2018-12-31,2174150.0,"11. Здание столовая на 108 мест, лит. Т пансио..."
251,"12. Здание проходной, лит. Ю по адресу: Камчат...",2018-01-01,2018-12-31,352360.0,"12. Здание проходной, лит. Ю по адресу: Камчат..."
252,"13. Здание проходной, лит. Э по адресу: Камчат...",2018-01-01,2018-12-31,359790.0,"13. Здание проходной, лит. Э по адресу: Камчат..."
253,"Всего по статье ""Текущий ремонт жилого фонда"" ...",2018-01-01,2018-12-31,159000.0,"Всего по статье ""Текущий ремонт жилого фонда"" ..."


In [11]:
save_dfs({"tr_joined": tr_joined}, path="parsed")
save_dfs({"ks_joined": ks_joined}, path="parsed")
save_dfs({"kr_joined": kr_joined}, path="parsed")

In [12]:
shutil.make_archive("parsed", 'zip', "parsed")

'/home/step/backend/parsed.zip'

In [13]:

def get_unique_keys(expenses_addresses, tr_joined, ks_joined, kr_joined, df_list_on):

    return set(itertools.chain(
        tr_joined["clean_address"],
        ks_joined["clean_address"],
        kr_joined["clean_address"],
        df_list_on["clean_address"],
        expenses_addresses["expenses_address"],
    ))


In [14]:
expenses_addresses = pd.read_excel("Обработанные адреса.xlsx")
df_list_on = pd.read_feather("data_lists_on.feather")

all_raw_strings = get_unique_keys(expenses_addresses, tr_joined, ks_joined, kr_joined, df_list_on)
len(all_raw_strings)

1785

In [15]:
def request_to_geocoder(string, api_key="8c1a744e-abae-4b1f-a292-076e70d90d92"):
    lat_1, lon_1 = 41.755060, 100.567964
    lat_2, lon_2 = 78.300151, -176.346714
    query = f"https://geocode-maps.yandex.ru/1.x?apikey={api_key}&geocode={string}&format=json&bbox={lon_1},{lat_1}~{lon_2},{lat_2}"
    res = requests.get(query)
    return json.loads(res.content)


def extract_lat_lon(json_response):
    try:
        lat, lon = map(float, json_response["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split())
        return lat, lon
    except Exception as exc:
        print(exc)
        return None, None


def extract_address(json_response):
    try:
        address = json_response["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["AddressDetails"]["Country"]["AddressLine"]
        return address
    except Exception as exc:
        print(exc)
        return None

def save_dict(path, dictionary):
    with open(path, 'wb') as f:
        pickle.dump(dictionary, f)

def read_dict(path):
    with open(path, 'rb') as f:
        return pickle.loads(f.read())


def get_appliable_function_and_geocoder_results(geocoder_results_path):
    geocoder_results = read_dict(geocoder_results_path)

    def get_geocoder_features(row):
        json_response = geocoder_results[row["raw_string"]]
        row["geocoder_address"] = extract_address(json_response)
        lat, lon = extract_lat_lon(json_response)
        row["geocoder_lat"] = lat
        row["geocoder_lon"] = lon
        return row

    return get_geocoder_features, geocoder_results


def get_geocoder_results(all_raw_strings):
    geocoder_results = dict()
    for raw_string in all_raw_strings:
        geocoder_results[raw_string] = request_to_geocoder(raw_string)
    return geocoder_results

In [16]:
# geocoder_results = get_geocoder_results(all_raw_strings)
# save_dict("geocoder_results.pkl", geocoder_results)

In [17]:
get_geocoder_features, geocoder_results = get_appliable_function_and_geocoder_results("geocoder_results.pkl")
df = pd.DataFrame()
df["raw_string"] = geocoder_results.keys()
df = df.apply(get_geocoder_features, axis=1)
df

'response'
'response'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of 

,raw_string,geocoder_address,geocoder_lat,geocoder_lon
0,,None,NaN,NaN
1,"город Хабаровск, р-н Центральный, ул. Мурав...","Россия, Хабаровск, улица Муравьёва-Амурского, 42",135.067820,48.477751
2,"Саха (Якутия), Алданский р-н, г. Алдан, 10 ...","Россия, Республика Саха (Якутия), Алдан, улица...",125.392453,58.606582
3,"г. Хабаровск, р-н Центральный, ул. Муравьев...","Россия, Хабаровск, улица Муравьёва-Амурского, 42",135.067820,48.477751
4,"г Владивосток, ул Верхнепортовая, д 44, кв 12","Россия, Приморский край, Владивосток, Верхнепо...",131.869890,43.103978
...,...,...,...,...
2260,"РКЦ Алдан, ул.10 лет Якутии, 31А -","Россия, Республика Саха (Якутия), Алдан, улица...",125.392453,58.606582
2261,"Хабаровск с. Троицкое, ул. Калинина, 95","Россия, Хабаровский край, Нанайский район, сел...",136.547418,49.444093
2262,"город Магадан, улица Пушкина, здание 4 (37-01-...","Россия, Магадан, улица Пушкина, 4",150.804651,59.566828
2263,"город Курильск, улица Сахалинская, дом 1 ко...","Россия, Сахалинская область, Курильск, Сахалин...",147.873826,45.231763
